In [1]:
import numpy as np
import random
import scipy
import pandas as pd
import hw4_functions
import algorithms
import math
import random
import time
import matplotlib.pyplot as plt

In [2]:
#calculate stochastic gradient
def svm_objective_function_stochastic(w, features, labels, order):
    n=len(labels)
    if order==0:
        # value = (TODO: value)
        value = 0
        for i in range(0, n):
            value = value + np.max((1 - np.double(features[i,:] * w * labels[i,:]), 0))
            value = np.double(value / n)
        return value
    
    elif order==1:
        # value = ( TODO: value )
        # subgradient = ( TODO: sungradient )
        # value = ( TODO: value )
        #value = math.inf
        value = float('inf')

        # subgradient = (TODO: sungradient)
        idx = random.sample(range(0, n), 1)
        
#         subgradient = np.asmatrix(np.zeros((d,1)))
        subgradient = np.asmatrix(np.zeros((np.size(w),1)))


        if (1 - np.double(features[idx,:]*w * labels[idx,:])) <= 0:
            subgradient = subgradient + 0
        else:

#             print('yesssssss', float(labels[idx,0]))
#             subgradient = subgradient - labels[idx,:]*(features[idx,:].T)

            subgradient = subgradient - float(labels[idx,0])*(features[idx,:].T)
                              
        return (value, subgradient)
    else:
        raise ValueError("The argument \"order\" should be 0 or 1")

In [3]:
#calculate full gradient
def svm_objective_function(w, features, labels, order):
    n=len(labels)
    if order==0:
        # value = ( TODO: value )
        value = 0
        for i in range(0, n):
            value = value + np.max((1 - np.double(features[i,:] * w * labels[i,:]), 0))
        value = np.double(value / n)
        return value

    elif order==1:
    
        value= float('inf')
        subgradient = np.asmatrix(np.zeros((np.size(w),1)))
        
        print('subgradient subgradient 1', subgradient.shape)
      
        
        for i in range(0, n):
#             if (1 - np.double(np.dot(features[i,:],w) * labels[i,:]) <= 0):
            if (1 - np.double(np.dot(features[i,:],w) * float(labels[i,:])) <= 0):
                subgradient = subgradient + 0
            else:

#                 print('subgradient subgradient 2', subgradient.shape)
                subgradient = subgradient - float(labels[i,0])*(features[i,:].T)
#                 print('subgradient subgradient 3', subgradient.shape)

        subgradient =(1/n)* subgradient
        
        return (value, subgradient)
    
    else:
        raise ValueError("The argument \"order\" should be 0 or 1")



In [4]:
def svrg(func,sgd,x0,step_size,n,d,max_epoch=10, m=0):
    
    if not isinstance(m,int) or m<=0:
        m=n
            
    if x0 is None:
        x=np.zeros((d,1))
    elif isinstance(x0,np.ndarray) and x0.shape == (d,1):
        x=x0.copy()
    else:
        raise ValueError('x0 must be a np array of size(d,)')
    
    # initialization
    values = []
    runtimes = []
    grad_count = []
    xs = []
    iterations = 0 
    
    x= np.asmatrix(x)
    
    for k in range(max_epoch):
   
        value, full_gradient = func(x,1)
        
#         sto_value, sto_gradient = sgd(x,1)

        value = np.double(value)
        full_gradient = np.matrix(full_gradient)
    
        x_tilde = x.copy()
        
#         print('x_tilde ',x_tilde.shape )
        for i in range(m):
        
            x_tilde_value, x_tilde_gradient = sgd(x_tilde,1)
        
            sto_value, sto_gradient = sgd(x,1)
            
            
#             print('x_tilde_gradient', x_tilde_gradient.shape)
#             print('sto_gradient', sto_gradient.shape)
#             print('full_gradient', full_gradient.shape)

            x -= step_size * (sto_gradient - x_tilde_gradient + full_gradient)

            
            v=func(x,0)
            values.append(v)
            xs.append(x)
        
    return (x,value,xs,values)
            
           

In [5]:
def Sto_gd(func,sgd,x0,step_size,n,d,max_epoch=100):
    # initialization
    values = []
    grad_count = 0
    xs = []
    iterations = 0 
    
#     print('what is x0 ', x0.shape)

    if x0 is None:
        x=np.zeros((d,1))
    elif isinstance(x0,np.ndarray) and x0.shape == (d,1):
        x=x0.copy()
    else:
        raise ValueError('x0 must be a np array of size(d,)')
    
    x=np.asmatrix(x)
    
    for k in range(max_epoch):
        sto_value,sto_gradient =  sgd(x,1)
        x-= step_size * sto_gradient
        
        xs.append(x)
        value = func(x,0)
        values.append(value)
                      
        iterations += 1
        grad_count +=1
    return(x,xs,value,values)
        

In [6]:
def main():
    step_size = 0.025
   
    # problem size
    n, d = 1000, 100

    # randomly generate training data
    x0 = np.zeros((d,1))#d*1 array
    
#     x0 = x0.reshape((x0.shape[0], 1))
#     print(x0.shape)
    
    features = np.random.randn(n, d)
    features = np.asmatrix(features)#x*d matrix
    x_true = np.random.randn(d)
    labels = np.asmatrix(np.sign(np.dot(features, x_true) + 0.1 * np.random.randn(n)))

    labels = labels.T #n*1 matrix
#     print(x0.shape)

    func = lambda w,order: svm_objective_function(w, features, labels, order)
    sgd = lambda w,order: svm_objective_function_stochastic(w, features, labels, order)
    
    x,value,xs,values_svrg = svrg(func,sgd, x0, step_size, n, d)
    x,value,xs,values_stogd = Sto_gd(func, sgd,x0, step_size, n, d)
    
    return values_svrg, values_stogd
    
    

In [8]:
out = main()

In [ ]:
values_svrg = out[0]
values_stogd = out[1]


In [ ]:

plt.plot(values_svrg, color = 'b')
